In [1]:
import pymongo
import pandas as pd
import itertools
import csv
import tqdm

In [2]:
%run filepaths.ipynb

In [3]:
client = pymongo.MongoClient()
db = client['uni']

In [21]:
def get_graduates(vpo1_collection):
    return vpo1_collection.aggregate(
        pipeline=[
    {
        '$match': {
            'section': '2.1.3'
        }
    }, {
        '$project': {
            'columns': 0
        }
    }, {
        '$unwind': {
            'path': '$rows', 
            'preserveNullAndEmptyArrays': False
        }
    }, {
        '$match': {
            'rows.values.2.colname': 'Код классификатора', 
            'rows.values.2.value': 1, 
            'rows.values.3.colname': {
                '$regex': 'Код направления.*'
            }, 
            'rows.values.4.colname': {
                '$regex': 'Выпуск фактический.*', 
                '$not': {
                    '$regex': '.*инвалиды.*'
                }
            }
        }
    }, {
        '$project': {
            'region': 1, 
            'funded_by': 1, 
            'time_involvement': 1, 
            'graduates': {
                '$arrayElemAt': [
                    '$rows.values', 4
                ]
            }, 
            'group': {
                '$arrayElemAt': [
                    '$rows.values', 2
                ]
            }, 
            'program': '$rows.name', 
            'program_code': {
                '$arrayElemAt': [
                    '$rows.values', 3
                ]
            }
        }
    }, {
        '$project': {
            'region': 1, 
            'funded_by': 1, 
            'time_involvement': 1, 
            'group': '$group.value', 
            'program': 1, 
            'program_code': '$program_code.value', 
            'n_graduates': '$graduates.value'
        }
    }
])

In [22]:
fieldnames = list(next(get_graduates(db.vpo1)).keys())
fieldnames.remove('_id')
# fieldnames.remove('group')
fieldnames

['region',
 'funded_by',
 'time_involvement',
 'program',
 'group',
 'program_code',
 'n_graduates']

In [23]:
with open(GRADUATES_FILE, 'w', newline='') as f:
    f.truncate(0)
    writer = csv.DictWriter(f, fieldnames=fieldnames, dialect='unix')
    writer.writeheader()
    for record in tqdm.tqdm(get_graduates(db.vpo1)):
        # record['program'] = '{}. {}'.format(record['group'], record['program'])
        for k in list(record.keys()):
            if k not in fieldnames:
                del record[k]
        writer.writerow(record)

21650it [00:01, 14757.30it/s]


In [24]:
df = pd.read_csv(GRADUATES_FILE)
df

,region,funded_by,time_involvement,program,group,program_code,n_graduates
0,Российская Федерация,Частные,аттестация экстернов,Информатика и вычислительная техника,1.0,09.03.01,44.0
1,Российская Федерация,Частные,аттестация экстернов,Психология,1.0,37.03.01,1.0
2,Российская Федерация,Частные,аттестация экстернов,Экономика,1.0,38.03.01,371.0
3,Российская Федерация,Частные,аттестация экстернов,Менеджмент,1.0,38.03.02,495.0
4,Российская Федерация,Частные,аттестация экстернов,Государственное и муниципальное управление,1.0,38.03.04,98.0
5,Российская Федерация,Частные,аттестация экстернов,Бизнесинформатика,1.0,38.03.05,8.0
6,Российская Федерация,Частные,аттестация экстернов,Юриспруденция,1.0,40.03.01,922.0
7,Российская Федерация,Частные,аттестация экстернов,Зарубежное регионоведение,1.0,41.03.01,6.0
8,Российская Федерация,Частные,аттестация экстернов,Реклама и связи с общественностью,1.0,42.03.01,6.0
9,Российская Федерация,Частные,аттестация экстернов,Журналистика,1.0,42.03.02,0.0


In [25]:
df[df['program'] == 'Математика'].head()

,region,funded_by,time_involvement,program,group,program_code,n_graduates
178,Российская Федерация,Всего,очная,Математика,1.0,01.03.01,463.0
457,Российская Федерация,Всего,очная,Математика,1.0,01.04.01,442.0
703,Российская Федерация,"Государственные, Муниципальные",аттестация экстернов,Математика,1.0,01.04.01,11.0
826,Российская Федерация,Всего,аттестация экстернов,Математика,1.0,01.04.01,11.0
1194,Российская Федерация,Всего,очно-заочная,Математика,1.0,01.03.01,18.0
